<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Audio_Deepfake_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Audio Deepfake Detection, Fake Calls, Spoofing, Fraud Calls and Voice Cloning Analysis for Defensice Forensics**

In [6]:
!pip install -q numpy librosa soundfile matplotlib IPython webrtcvad pydub noisereduce pyAudioAnalysis speechbrain langchain openai langgraph transformers vllm requests ipywidgets
!pip install -q audiomentations hmmlearn eyed3 langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [8]:
# Import necessary libraries
import os
import json
import asyncio
import concurrent.futures
import requests
from datetime import datetime
from typing import List, Dict, Any, Tuple

import numpy as np
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
import IPython.display as ipd

import webrtcvad
from pydub import AudioSegment
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import noisereduce as nr

from pyAudioAnalysis import audioSegmentation as aS, ShortTermFeatures
import speechbrain as sb
from speechbrain.inference.speaker import SpeakerRecognition, EncoderClassifier

# For forensic report generation using LangChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import openai

# For LangGraph orchestration in report aggregation
from langgraph.graph import StateGraph, END

# For vLLM integrations with audio models
from transformers import AutoTokenizer
from vllm import LLM, EngineArgs, SamplingParams

# For UI components and Mermaid.js visualization in Colab
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML